In [12]:
import os
import pandas as pd
import numpy as np


In [13]:
# Load dataset
dfpath = os.path.join(os.getcwd(), "data", "filtered_long.csv")
print("Loading file from: {}".format(dfpath))

df = pd.read_csv(dfpath, header=0)
df.session = df.session.map(lambda x: x.strip("()").split(','))
df.head()

Loading file from: C:\Programming\CustomerAnalysis\GroupAssignment\data\filtered_long.csv


,Unnamed: 0,session,purchase,len,add
0,0,"[1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1...",0,18,2
1,1,"[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2...",0,139,18
2,2,"[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1...",0,41,16
3,3,"[1, 2, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1...",0,16,2
4,4,"[1, 1, 1, 1, 2, 3, 4, 1, 2, 1, 1, 1...",1,29,5


In [14]:
# Split set into training and validation set
from sklearn.model_selection import train_test_split

# No real reason for the selected ratio. In PS they used 70/30. Don't forget to set state.
training_set, eval_set = train_test_split(df, test_size=0.2, random_state=123, shuffle=False)
eval_set.head()

,Unnamed: 0,session,purchase,len,add
129080,129080,"[1, 1, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1...",0,17,6
129081,129081,"[1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1...",0,16,14
129082,129082,"[1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1...",0,38,19
129083,129083,"[1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1...",1,13,5
129084,129084,"[1, 1, 2, 3, 1, 1, 2, 3, 1, 1, 2, 1...",1,20,3


In [15]:
import numpy as np

def ngram_featurizer(session, n):
    
    """takes in a list and an integer defining the size of ngrams.
     Returns the ngrams of desired size in the input string"""
    session = ['#']*(n-1) + session + ['+']*(n-1)
    ngrams = [tuple(session[i:i+n]) for i in range(len(session)-n+1)]
    
    return ngrams
    

def encode_sessions(sessions, n, mapping=None):
    
    """
    Takes in a list of lists, an integer indicating the character ngrams' size,
    and a dictionary mapping ngrams to numerical indices. If no dictionary is passed,
    one is created inside the function.
    The function outputs a 2d NumPy array with as many rows as there are strings in 
    the input list, and the mapping from ngrams to indices, representing the columns 
    of the NumPy array.
    """
    
    if not mapping:
        all_ngrams = set()
        for session in sessions:
            all_ngrams = all_ngrams.union(set(ngram_featurizer(session, n)))
    
        mapping = {ngram: i for i, ngram in enumerate(all_ngrams)}
    
    X = np.zeros((len(sessions), len(mapping)))
    for i, session in enumerate(sessions):
        for ngram in ngram_featurizer(session, n):
            try:
                X[i, mapping[ngram]] += 1
            except KeyError:
                pass
    
    return X, mapping

In [43]:
def subset(data, clicks):
    data = data[(data['len'] - data['add']) > clicks]
    data.session = data.apply(lambda x: x.session[:(x['add'] + clicks + 1)], axis=1)
    return data

train = subset(training_set, 10)
test = subset(eval_set, 10)

C:\Programming\Anavonda\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Programming\Anavonda\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [36]:
Xtrain, mapping = encode_sessions(train.session, 5)
Xtest, _ = encode_sessions(test.session, 5, mapping)

In [39]:
# fit NB classifier (Complement)
from sklearn.naive_bayes import ComplementNB

NB = ComplementNB(alpha=1, fit_prior=True)
NB.fit(Xtrain, train.purchase)
bayes_predictions = NB.predict(Xtest)

In [37]:
# fit NB classifier
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB(alpha=1, fit_prior=True)
NB.fit(Xtrain,train.purchase)
bayes_predictions = NB.predict(Xtest)

In [19]:
from sklearn import metrics

#acc = metrics.accuracy_score(test.purchase.to_list(), bayes_predictions)
#acc

In [40]:
f1 = metrics.f1_score(test.purchase.to_list(), bayes_predictions)
f1

0.45007812106244643

In [ ]:
majority = 1 - np.mean(test.purchase.to_list())
majority

In [34]:
print(Xtest)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [47]:
import math

# World's shittiest Markov chain

def transmat(sessions):
    n = 5
    M = [[0]*n for _ in range(n)]
    
    for session in sessions:
        for (i, j) in zip(session, session[1:]):
            M[int(i)][int(j)] += 1
            
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]

    return M

buy_matrix = transmat(train.session[train['purchase'] == 1])
ws_matrix  = transmat(train.session[train['purchase'] == 0])

preds = []
for session in test.session:
    buy_score = 0
    ws_score  = 0
    for i, w in enumerate(session, start = 1):
        buy_score += buy_matrix[int(session[i-1])][int(w)]
        ws_score  += ws_matrix[int(session[i-1])][int(w)]
    preds.append([1 if buy_score >= ws_score else 0])

f1 = metrics.f1_score(test.purchase.to_list(), preds)

[0.0, 0.6556666639073533, 0.2619968047151147, 0.047726463746761254, 0.034610067630770756]
[0.0, 0.615635145837047, 0.31767958222895, 0.04607433729693714, 0.020610934637065783]


In [45]:
print(M)

NameError: name 'M' is not defined